# lorapy data processing

In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import os, sys
from loguru import logger
logger.remove(None)
logger.add(sys.stdout, colorize=True)

import numpy as np

import matplotlib.pyplot as plt 
plt.style.use('dark_background')
%matplotlib inline

In [2]:
# from lorapy.io.load import DatLoader
import lorapy

## setup 

In [3]:
DATA_DIR = pathlib.Path('/Users/trevor/Work/1-projects/lora-processing/data/lora-benchtop-recording')

## utils

In [4]:
import random
rand = lambda mx: random.randint(0, mx)

# load

In [5]:
loader = lorapy.load(DATA_DIR, autoload=True)
loader.filelist

2020-04-05 12:54:38.519 | DEBUG    | lorapy.io.load:_process_data_dir:68 - set datafile directory: /Users/trevor/Work/1-projects/lora-processing/data/lora-benchtop-recording
2020-04-05 12:54:38.525 | INFO     | lorapy.io.load:_process_data_dir:74 - found 110 datafile file(s)


[DatFile('lora_BW2_SF10_915MHz_20ksps_Att100_v0.dat'),
 DatFile('lora_BW2_SF12_915MHz_20ksps_Att80_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att0_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att40_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att60_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att120_v0.dat'),
 DatFile('lora_BW2_SF12_915MHz_20ksps_Att20_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att40_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att60_v0.dat'),
 DatFile('lora_BW2_SF12_915MHz_20ksps_Att0_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att100_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att120_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att20_v0.dat'),
 DatFile('lora_BW2_SF12_915MHz_20ksps_Att60_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att140_v0.dat'),
 DatFile('lora_BW2_SF12_915MHz_20ksps_Att100_v0.dat'),
 DatFile('lora_BW2_SF11_915MHz_20ksps_Att80_v0.dat'),
 DatFile('lora_BW2_SF10_915MHz_20ksps_Att80_v0.dat'),
 DatFile('lora_BW2_SF12_

In [6]:
for file in loader.filelist:
    break
    
file

DatFile('lora_BW2_SF10_915MHz_20ksps_Att100_v0.dat')

In [7]:
tar_filename = 'lora_BW9_SF11_915MHz_20ksps_Att20_v0.dat'

file = [file for file in loader.filelist if file.name == tar_filename][0]
file

DatFile('lora_BW9_SF11_915MHz_20ksps_Att20_v0.dat')

# process

## convert 

In [8]:
signal = file.to_signal()
signal

2020-04-05 12:54:38.643 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 82 and packet length: 2480
2020-04-05 12:54:38.712 | INFO     | lorapy.datafile.file:load:31 - loaded 15975424 samples from file


LoraSignal(length=15975424)

In [11]:
signal.stats

BW: 0 | SF: 0 | Att: 0 | samples per symbol: 0 | packet length: 0

## extract packets

In [10]:
signal.extract_packets(method='slide-mean', auto_adj=False)

2020-04-05 12:54:38.786 | INFO     | lorapy.signals.signal:_process_signal:118 - selected "slide-mean" processing method
2020-04-05 12:54:38.835 | INFO     | lorapy.signals.processing.sliding_mean:_find_all_mindices:55 - scanning signal for padding locations..
2020-04-05 12:54:38.836 | DEBUG    | lorapy.signals.processing.sliding_mean:_find_all_mindices:59 - iteration 0


ValueError: attempt to get argmin of an empty sequence

In [ ]:
plt.plot(signal._raw_packets[rand(signal.num_packets), :])

## inspect packets

In [ ]:
for packet in signal.packets:
    break
    
packet

In [ ]:
packet.endpoints

In [ ]:
packet.plot()

In [ ]:
packet.adjustment

### random packet

In [ ]:
packet = signal.packets[rand(signal.num_packets)]
packet.plot()

In [ ]:
start, stop = packet.endpoints
start, stop

In [ ]:
plt.plot(signal.real_abs_signal[start-600:stop])

## symbol test

In [ ]:
numSymbols = 8
symbols = np.empty([signal.num_packets * numSymbols, int(signal.stats.samp_per_sym)], dtype='complex')
N = symbols.shape[0]

k,l = 0, 0

for i in range(signal.num_packets):
    for j in range(numSymbols):
        
        symbols[l,:] = signal._raw_packets[i, k:k+signal.stats.samp_per_sym]
        k = k + signal.stats.samp_per_sym
        l = l+1
    k = 0
    
print("Total Symbols: " + str(symbols.shape[0]))

In [ ]:
plt.plot(symbols[rand(symbols.shape[0]), :])